<a href="https://colab.research.google.com/github/Ssevinc/assistive-vision-ai/blob/main/train/train_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8 Training on Custom Assistive Vision Dataset

This notebook trains a YOLOv8 object detection model on a custom dataset of street signs, traffic lights, crosswalks, and other safety-related objects to assist visually impaired individuals. The dataset was created using [Roboflow](https://roboflow.com) and exported in YOLOv8 format.


## 1. Install Dependencies
We install:
- `ultralytics` → YOLOv8 training framework
- `roboflow` → to download the annotated dataset directly from Roboflow


In [ ]:
!pip install roboflow ultralytics

## 2. Download Dataset
We use Roboflow's API to download the dataset in YOLOv8 format.

The dataset contains the following classes:
1. bus_stop
2. crosswalk_sign
3. downstair_sign
4. no_entry
5. restroom
6. slippery_sign
7. stairs
8. stop
9. upstair_sign


In [3]:
from roboflow import Roboflow

# Enter your Roboflow API key here
rf = Roboflow(api_key=input("Enter your Roboflow API key: "))

project = rf.workspace("seyyide").project("dataset-z6sm4")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dataset-1 in yolov8:: 100%|██████████| 2698/2698 [00:00<00:00, 5589.47it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 3. Verify GPU Availability
YOLOv8 training on a CPU is slow. We check if Colab is using an NVIDIA GPU (Tesla T4 in this case) to ensure faster training.
(Make sure to change the runtime type to T4 GPU)

In [4]:
!nvidia-smi

Wed Aug 13 13:59:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 4. Train YOLOv8 Model
We use transfer learning from `yolov8n.pt` (nano model) to train on our dataset for 70 epochs.
- `imgsz=640` → resize images for speed & accuracy balance
- `epochs=70` → chosen to avoid overfitting (tested and observed in Roboflow experiments)
- `data.yaml` → dataset configuration file from Roboflow


In [ ]:
!yolo detect train model=yolov8n.pt data=/content/dataset-1/data.yaml epochs=70 imgsz=640

## Model Performance

The YOLOv8 Nano (`yolov8n.pt`) model was trained on our custom assistive vision dataset for 70 epochs with an image size of 640×640. The best model checkpoint (`best.pt`) achieved the following performance on the validation set:

| Class           | Precision (P) | Recall (R) | mAP50 | mAP50-95 |
|-----------------|--------------:|-----------:|------:|---------:|
| **bus_stop**        | 0.980 | 1.000 | 0.995 | 0.977 |
| **crosswalk_sign**  | 0.985 | 0.962 | 0.987 | 0.949 |
| **downstair_sign**  | 0.833 | 0.681 | 0.796 | 0.335 |
| **no_entry**        | 0.985 | 1.000 | 0.995 | 0.990 |
| **restroom**        | 0.984 | 1.000 | 0.995 | 0.965 |
| **slippery_sign**   | 0.940 | 0.957 | 0.932 | 0.927 |
| **stairs**          | 0.988 | 1.000 | 0.995 | 0.975 |
| **stop**            | 0.971 | 1.000 | 0.995 | 0.974 |
| **upstair_sign**    | 0.656 | 0.481 | 0.529 | 0.202 |

**Overall Performance:**
- **mAP50:** 0.913  
- **mAP50-95:** 0.810  

### 💡 Insights
- Most classes achieved **very high precision and recall**, with near-perfect detection on `bus_stop`, `stairs`, `stop`, and `restroom`.
- `downstair_sign` and `upstair_sign` have significantly lower recall and mAP, indicating the model sometimes misses these signs or has difficulty with precise localization.
- To improve performance on weaker classes:
  - Collect more training images for those categories.
  - Add targeted data augmentation (rotations, perspective changes).
  - Consider fine-tuning with class-weighted training.

---

**Trained Weights:** [`best.pt`](models/best.pt)  


This cell is to download the trained weights from Colab so they can be moved into the models/ folder in the GitHub repo.

In [6]:
# @title Download the model
from google.colab import files

files.download('/content/runs/detect/train/weights/best.pt')
files.download('/content/runs/detect/train/weights/last.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>